In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import (
    Flatten,
    Dense,
    Dropout,
    BatchNormalization,
    LeakyReLU,
)
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

import numpy as np
import os

# 加载MNIST数据集
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 数据预处理
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

x_train = np.expand_dims(x_train, axis=-1)  # 增加通道维度
x_test = np.expand_dims(x_test, axis=-1)

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 数据增强
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    zoom_range=0.5,
    shear_range=0.5,
    fill_mode="nearest",
)
datagen.fit(x_train)

# 构建 BP 神经网络模型
# 没有池化 池化操作主要应用在CNN网络中
model = Sequential(
    [
        Flatten(input_shape=(28, 28, 1)),
        
        Dense(1024),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(1024),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        # ----------------- 两次完全一样的 提取特征 -----------------
        
        Dense(512),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(256),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(128),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(64),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(32),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        # 输入结果
        Dense(10, activation="softmax"),
    ]
)

# 编译模型
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), # 使用 Adam 优化器
    loss="categorical_crossentropy",  # 多分类交叉熵损失
    metrics=["accuracy"],
)

# 回调函数
reduce_lr = ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=10, min_lr=1e-6, verbose=1
)
early_stopping = EarlyStopping(
    monitor="val_loss", patience=15, restore_best_weights=True, verbose=1
)

# 训练模型
model.fit(
    datagen.flow(x_train, y_train, batch_size=64),
    epochs=1000,
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // 64,
    callbacks=[reduce_lr, early_stopping],
)

# 评估模型
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc}")

# 保存模型
model.save("mnist_model_bp_ubuntu.keras")

2025-01-14 15:53:02.474044: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2025-01-14 15:53:04.165642: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2025-01-14 15:53:04.166613: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2025-01-14 15:53:04.172915: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:af:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.75GiB deviceMemoryBandwidth: 573.69GiB/s
2025-01-14 15:53:04.172952: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2025-01-14 15:53:04.174560: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.

Epoch 1/1000


2025-01-14 15:53:06.340705: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


937/937 [==============================] - 31s 32ms/step - loss: 2.7371 - accuracy: 0.1246 - val_loss: 2.0129 - val_accuracy: 0.2285
Epoch 2/1000
937/937 [==============================] - 28s 30ms/step - loss: 2.1557 - accuracy: 0.2047 - val_loss: 1.8818 - val_accuracy: 0.2781
Epoch 3/1000
937/937 [==============================] - 29s 31ms/step - loss: 2.0657 - accuracy: 0.2376 - val_loss: 1.6180 - val_accuracy: 0.4129
Epoch 4/1000
937/937 [==============================] - 30s 32ms/step - loss: 1.9658 - accuracy: 0.2874 - val_loss: 1.3465 - val_accuracy: 0.5351
Epoch 5/1000
937/937 [==============================] - 29s 31ms/step - loss: 1.8663 - accuracy: 0.3261 - val_loss: 1.2081 - val_accuracy: 0.5332
Epoch 6/1000
937/937 [==============================] - 32s 34ms/step - loss: 1.7953 - accuracy: 0.3595 - val_loss: 1.1036 - val_accuracy: 0.6150
Epoch 7/1000
937/937 [==============================] - 33s 35ms/step - loss: 1.7421 - accuracy: 0.3801 - val_loss: 1.0852 - val_accuracy

### BP网络中 反向传播是如何体现的 ?

在 Keras 中，反向传播是由框架自动处理的。你不需要显式地编写反向传播的代码。反向传播的过程在模型编译和训练过程中自动完成。以下是代码中体现反向传播的部分：

1. **模型编译**：
在编译模型时，指定了优化器和损失函数。优化器（如 Adam）会在训练过程中自动执行反向传播，以更新模型的权重。

2. **模型训练**：
在调用 `model.fit` 方法时，Keras 会自动执行前向传播和反向传播，以最小化损失函数。



1. **前向传播**：
输入数据通过网络的各层，计算输出和损失。

2. **计算损失**：
使用指定的损失函数（如 categorical_crossentropy）计算预测值与真实值之间的差异。

3. **反向传播**：
计算损失相对于每个参数的梯度。
优化器（如 Adam）使用这些梯度更新模型的权重。

4. **权重更新**：
根据计算的梯度和学习率，优化器调整模型的权重，以最小化损失。



通过这些步骤，Keras 自动处理反向传播的过程，更新模型的权重，以最小化损失函数。

#### 与卷积神经网络的不同点 ?

在传统的 BP 神经网络（全连接层）中，通常不使用池化操作。池化操作主要用于卷积神经网络（CNN）中，用于减少特征图的尺寸和计算量，同时保留重要的特征。

### 为什么 BP 网络中不需要池化：

1. **全连接层的特点**：
全连接层中的每个神经元与前一层的所有神经元相连接，因此不需要通过池化来减少特征图的尺寸。

2. **池化的作用**：
池化操作（如 MaxPooling）主要用于卷积层之后，用于减少特征图的尺寸，降低计算量，并提高模型的平移不变性。

